In [3]:
from dotenv import load_dotenv 
load_dotenv()
import os
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI

api_key = os.environ.get("PINECONE_API_KEY")


/Users/sam/miniconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec(cloud='aws', region='us-east-1')
pc.create_index(
        name='forest-rag',
        dimension=1536,
        metric="cosine",
        spec=spec
    )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '55eda3e7af254b1ba3a34ceb962f2672', 'Date': 'Thu, 22 Aug 2024 06:57:09 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open('amazon.json'))
data['amazon']

In [6]:
processed_data = []
client = OpenAI()
for amazon in data['amazon']:
    response = client.embeddings.create(
        input = amazon['description'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": amazon["species"],
        "metadata": {
            "scientific_name": amazon["scientific_name"],
            "category": amazon["category"],
            "description": amazon["description"],
            "traits": amazon["traits"]
        }
    })

In [58]:
processed_data = []
client = OpenAI()
for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["state"],
        "metadata": {
            "year": review["year"],
            "description": review["description"],
            "link": review["link"]
        }
    })

In [19]:
index = pc.Index('forest-rag')
index.upsert(
    vectors = processed_data,
    namespace='ns1'
)

{'upserted_count': 55}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 32}},
 'total_vector_count': 32}